# Import Solidity Source Code

In [ ]:
f = open("uts.sol", "r")
file_content = f.read()
f.close()

#print(file_content)

In [1]:
# For testing
file_content = """
// SPDX-License-Identifier: GPL-3.0
pragma solidity ^0.4.24;

contract something {
  uint public coolade;
  address public checker;
  address public owner;
  mapping(address => bool) public whitelist;
  int public more = 0;
  int256 public more3=23;
  int8 more2 =0;
  uint16 more4;
  uint8 more5=0;
  uint[4] coolade2;
  int[][2] coolade3;
  int[2] coolade4;
  address[] fishe;
  string[] arr;
  bool bool1 = false;
  bool bool2 = true;
  string str1 = "";
  string str2 = "aa";
  string str3 = '';
  address address1 = 0x0000000000000000000000000000000000000000;
  address address2 = 0x0000000000000000000000000000000000000123;
  bytes4 bytesVar1 = "";
  bytes4 bytesVar2 = "abcd";
  bytes4 bytesVar3 = 0;
  bytes4 public bytesVar4 = '';
  int bool3;
  int more6 = more3++;
  int more10 = more9 - 1;  // 7
  int more7 = 2 + 3;  // 5
  int more8 = more7 + 1;  // 6
  int more9 = (more8 - 2) * 2;  // 8
  int more11 = more6 / 2;

  event SetPrice1(uint256 price, address a);
  event SetPrice2(bytes32 price, string b);
  event SetPrice3(int256 price, string indexed c);
  event SetPrice4(uint256 indexed price, address indexed a);  // no change

  constructor() public {
    owner = msg.sender;
  }

  modifier onlyOwner() {
    require(msg.sender == owner);
    _;
  }

  function setChecker(address _checker) public onlyOwner {
    checker = _checker;
    for (int i = 0; i<10; i++) {
      more++;
    }

    //for (int i=0;i<5;i++) {  // <- Should give a space between code operations or it won't work.
    //  more ++;  // <- Increment should connect with the variable name or it won't work.
    //}

    bool3 = ++more;  // Okay to be replaced (optimal)
    bool3 = more++;  // Must not be replaced 1
    bool3=more++;  // Must not be replaced 2
    bool3 = more3 + more++;  // Must not be replaced 3
    bool3=more3+more++;  // Must not be replaced 4

    bool3 = 0;
    bool3=0;

    if (bool3 == 0) {
    }

    if (bool1 == true) {
    }

    if (true == true && false) {
    }

    if (true || false == true && false) {}

    if (true || false == (true)) {}

    if (true == true) {}

    if ((true == true) == (true == true) == (true == true == true)) {}

    if (true == true == true == true) {}

    if ((true == true) == (true == true) == (true == true == true) == (true == true == true == true)) {}

    if (true == true == true == true == true == true == true) {}

    if ((true == true) == (true == false) == (true == false == true) == (true == false == true == true)) {}

    if (false == true == true == false == true == false == true) {}

    if (((((true)))) == (((true))) == (true) == true) {}
    if (((((true)))) == (((true))) == (true) == false) {}
    if (((((true)))) == (((true))) == (false) == true) {}
    if (((((true)))) == (((true))) == (false) == false) {}

  }

  function approve(address _wallet) public onlyOwner {
    whitelist[_wallet] = true;
  }
}
"""
# Deployment: 676762 vs 649643  (original vs optimized)  [27119 diff (~4%)]
# approve: 46238 vs 46238  (original vs optimized)  [0 diff]
# setChecker: 79516 vs 64159  (original vs optimized)  [15357 diff (~19.3%)]

#print(file_content)

In [2]:
import re

# Get contract version
#version = re.search('pragma solidity \^?(.*);', file_content).group(1)
# Get contract names
#name = re.findall('contract (.*) {', file_content)

# Utility functions
def swapPositionList(lst, pos1, pos2):  # Not used
  lst[pos1], lst[pos2] = lst[pos2], lst[pos1]
  return lst

def swapPositionDict(dct, key1, key2):  # Not used
  dct[key1], dct[key2] = dct[key2], dct[key1]
  return dct

def bitToByte(bit):
  return bit/8

def bestFit(processSize):
  formattedProcessSize = []

  # Format the processSize to fit with the format that can be used for final optimization
  for i in processSize:
    for j in processSize[i]:
      j["size"] = i
      formattedProcessSize += [j]

  n = len(formattedProcessSize)

  # Gerenate all page containers. All is set to 32 bytes by default (its maximum possible partition)
  blockSize = []
  for i in range(n):
    blockSize += [32]

  # Stores block id of the block allocated to a process
  allocation = {}

  # pick each process and find suitable blocks according to its size and assign to it
  for i in range(n):

    # Find the best fit block for current process
    bestIdx = -1
    for j in range(n):
      if blockSize[j] >= formattedProcessSize[i]["size"]:
        if bestIdx == -1:
          bestIdx = j
        elif blockSize[bestIdx] > blockSize[j]:
          bestIdx = j

    # If we could find a block for current process
    if bestIdx != -1:

      # allocate block j to process[i] process
      allocation.setdefault(bestIdx, []).append(formattedProcessSize[i])

      # Reduce available memory in this block.
      blockSize[bestIdx] -= formattedProcessSize[i]["size"]

  #print("blockSize:", blockSize)
  #print("processSize:", processSize)
  #print("allocation:", allocation)

  return allocation

In [9]:
### Process:
# Seperate source code to each lines -> list
# Scan and mark each lines based on its characteristics -> dict
# Optimize source code
# Combine all source code lines back

### Main Options
# https://www.geeksforgeeks.org/best-fit-allocation-in-operating-system/
VARIABLE_PACKING = True  # This will only sort the variables byte size from big to small.
VARIABLE_PACKING_BEST_FIT = True  # Variable Packing variable needs to be turned on for this to work. (Decreases 16 gas from test SC)
if VARIABLE_PACKING_BEST_FIT == True:
  VARIABLE_PACKING = True
DEFAULT_VALUE = True
INCREMENT_OPERATOR = True
DECREMENT_OPERATOR = True
PRECOMPUTE_VARIABLE_VALUE = True  # Unreference variables and precompute them if not used for increment/decrement or functions.
BOOLEAN_IF_CHECK = True
INDEX_EVENT_VARIABLE = True

### Miscellaneous Options (May increase gas cost if set to True)
PRECOMPUTE_VALUE = False  # Lazy check on every " = " and see if it can be evaluated with Python. Address is ignored.
INDEX_EVENT_STRING_BYTES = False  # Allow string and bytes variable to be available for indexing. (increases 12 gas from test SC)
if INDEX_EVENT_STRING_BYTES == True:
  INDEX_EVENT_VARIABLE = True

### Variable unpacking
# Scan for all variable islands
# Sort each variable islands from big to small byte size

### Seperate source code to each lines -> list
code_lines = file_content.split("\n")
code_metadata = {}

# Variables for detecting issues
all_vars = []  # Stores all islands of vars. Content: dict of byte sizes group
current_island_var = -1
last_index_var = -1  # To detect when the islands cut off.  if detected var > last_index_var+1
sorted_var = []

# 0: Symbol opening/closing, 1: major opener, 2: default value issue, 3: data types, 4: visibility, 5: operator, 6: error handling
# Remember the ordering. First come, first serve
tags = {0: "{",
        1: "}",
        100: "contract ",
        101: " function ",
        102: " modifier ",
        103: " constructor",
        104: " if( +)?\(",  # Then checks for inefficient boolean
        105: " event(.*?)\(",  # Then checks for indexed variables in event
        201: "bool(.*?)( ?)=( ?)false",  # Default Values (bool)
        202: "(u?)int(.*?)( ?)=( ?)0",  # Default Values (int)
        203: "string(.*?)( ?)=( ?)(('')|(\"\"))",  # Default Values (string)
        204: "address(.*?)( ?)=( ?)0x0000000000000000000000000000000000000000",  # Default Values (address)
        205: "bytes(.*?)( ?)=( ?)((0(x(0+))?)|('')|(\"\"))",  # Default Values (bytes)
        301: " bool((\[([0-9]+)?\])+)? ",
        302: " int([0-9]+)?((\[([0-9]+)?\])+)? ",  # old: " int(.*?) "
        303: " uint([0-9]+)?((\[([0-9]+)?\])+)? ",
        304: " bytes([0-9]+)?((\[([0-9]+)?\])+)? ",
        305: " address((\[([0-9]+)?\])+)? ",
        306: " mapping\(",  # Variable length
        307: " string((\[([0-9]+)?\])+)? ",  # Variable length
        401: " public ",
        402: " private ",
        403: " external ",
        501: " for( |\()?",
        502: "\+\+( |\)|;)", # Should not include for "= x++" or math operations or conditionals as it may break SC functionality  # \+\+( |\)|;|<|>|=|\||\&|\+|\-|\*|/)
        503: "--( |\)|;)",
        504: "( +)?=( +)?",  # For precompute
        601: " require\(",
        602: " revert\(",
        603: " assert\(",  # No error message
        }  # token: tag

def add_variable_island(byte_size, dict_line):
  global last_index_var
  global current_island_var
  global all_vars
  global tags

  # Ignore if it's a function / modifier / similar types
  ignore_case = (101, 102, 103, 105)
  for i in ignore_case:
    if re.search(tags[i], dict_line["value"]) != None:
      return

  # Start new island if not close to an island
  if last_index_var+1 < dict_line["index"]:
    current_island_var += 1
    all_vars.append({})
    all_vars[current_island_var]["metadata"] = {}
    all_vars[current_island_var]["metadata"]["start_index"] = dict_line["index"]
    all_vars[current_island_var]["metadata"]["total_line"] = 0
    all_vars[current_island_var]["metadata"]["byte_sizes"] = set()
    all_vars[current_island_var]["metadata"]["variables"] = {}

  # Add to island var if not added before
  if last_index_var != dict_line["index"]:  # Avoid adding the same line twice
    all_vars[current_island_var].setdefault(byte_size, []).append(dict_line)
    all_vars[current_island_var]["metadata"]["total_line"] += 1
    all_vars[current_island_var]["metadata"]["byte_sizes"].add(byte_size)
    if dict_line["var_name"] != None:
      all_vars[current_island_var]["metadata"]["variables"][dict_line["var_name"]] = dict_line["var_value"]
    last_index_var = dict_line["index"]

def get_var_name_value(code_line, default_value = "0"):
  # Doesn't work on mapping
  if "=" not in _line_data["value"]:
    value = default_value
    left_side = re.sub("( +)?;(.*$)?", "", code_line)  # Get the left side of ";", then strip any edge whitespaces
    name = left_side.split(" ")[-1]  # Variable name should be on the right-most of the string
  else:
    clean = re.sub("^(.*?)=( +)?|( +)?;(.*$)?", "", code_line)  # Delete "...= " and " ;..."
    value = clean
    left_side = re.sub("( +)?=(.*$)?", "", code_line)  # Delete " =..."
    name = left_side.split(" ")[-1]  # Variable name should be on the right-most of the string
  return value, name


### Scan and mark each lines based on its characteristics -> dict
for index, value in enumerate(code_lines):
  metadata = [value]
  for token in tags:

    # https://pynative.com/python-regex-pattern-matching/
    # Check each code line for tag tokens
    if re.search(tags[token], value) != None:
      metadata += [token]
      _line_data = {"index": index, "value": value, "token": token}

      ## Default value fix
      if DEFAULT_VALUE == True:
        if token == 201:
          print(index, "index, default value bool issue found!!", value)

          fixed = re.sub("( ?)=( ?)false", "", value)
          #print("Fixed to:", fixed)
          value = fixed
          metadata[0] = fixed
          code_lines[index] = fixed
        elif token == 202:
          print(index, "index, default value int issue found!!", value)

          fixed = re.sub("( ?)=( ?)0", "", value)
          #print("Fixed to:", fixed)
          value = fixed
          metadata[0] = fixed
          #_line_data["value"] = fixed
          code_lines[index] = fixed
        elif token == 203:
          print(index, "index, default value string issue found!!", value)

          fixed = re.sub("( ?)=( ?)(('')|(\"\"))", "", value)
          #print("Fixed to:", fixed)
          value = fixed
          metadata[0] = fixed
          code_lines[index] = fixed
        elif token == 204:
          print(index, "index, default value address issue found!!", value)

          fixed = re.sub("( ?)=( ?)0x0000000000000000000000000000000000000000", "", value)
          fixed = re.sub("( ?)=( ?)address\(0\)", "", fixed)
          #print("Fixed to:", fixed)
          value = fixed
          metadata[0] = fixed
          code_lines[index] = fixed
        elif token == 205:
          print(index, "index, default value bytes issue found!!", value)

          fixed = re.sub("( ?)=( ?)((0(x(0+))?)|('')|(\"\"))", "", value)
          #print("Fixed to:", fixed)
          value = fixed
          metadata[0] = fixed
          code_lines[index] = fixed

      ## Unoptimize operator use
      if INCREMENT_OPERATOR == True:
        if token == 502:  # x++
          print(index, "index, potentional unoptimal x++ use detected!", value)

          # Get variable name
          left_wall_var = " |\(|;"  # " |\(|\)|;|<|>|=|\||\&|\+|\-|\*|/"  # " |\(|\)|;"
          prohibited_chars = (">", "<", "=", "|", "&", "+", "-", "*", "/")
          line_split = value.split("++")

          for i in range(len(line_split)-1):
            # Cut from the right edge until hits the left wall
            var_splits = re.split(left_wall_var, line_split[i])
            left_var_name = var_splits[-2]
            var_name = var_splits[-1]

            #print("left_var_name:", left_var_name, "| var_name", var_name)

            # Check variable name and its left to not contain prohibited chars
            if (len(left_var_name) != 0 and any(x in left_var_name for x in prohibited_chars)) or any(x in var_name for x in prohibited_chars):
              print("Prohibited char detected! Continued to next line.")

            elif var_name == "":
              print("Empty variable found")

            else:
              fixed = value.replace(var_name+"++", "++"+var_name)

              value = fixed
              metadata[0] = fixed
              code_lines[index] = fixed
              #print("Fixed to:", fixed)

      if DECREMENT_OPERATOR == True:
        if token == 503:  # x--
          print(index, "index, potentional unoptimal x-- use detected!", value)

          # Get variable name
          left_wall_var = " |\(|;"  # " |\(|\)|;|<|>|=|\||\&|\+|\-|\*|/"  # " |\(|\)|;"
          prohibited_chars = (">", "<", "=", "|", "&", "+", "-", "*", "/")
          line_split = value.split("--")

          for i in range(len(line_split)-1):
            # Cut from the right edge until hits the left wall
            var_splits = re.split(left_wall_var, line_split[i])
            left_var_name = var_splits[-2]
            var_name = var_splits[-1]

            #print("left_var_name:", left_var_name, "| var_name", var_name)

            # Check variable name and its left to not contain prohibited chars
            if (len(left_var_name) != 0 and any(x in left_var_name for x in prohibited_chars)) or any(x in var_name for x in prohibited_chars):
              print("Prohibited char detected! Continued to next line.")

            elif var_name == "":
              print("Empty variable found")

            else:
              fixed = value.replace(var_name+"--", "--"+var_name)

              value = fixed
              metadata[0] = fixed
              code_lines[index] = fixed
              #print("Fixed to:", fixed)

      ## If check boolean
      if BOOLEAN_IF_CHECK == True:
        if token == 104:
          print(index, "index, if check detected", value)

          boolean_clean = False  # If True, no other inefficiencies found

          while boolean_clean == False:
            boolean_clean = True

            # Check inefficient bool check for the right side
            statement = re.search(r'( +)?==( +)?(\(+)?( +)?true( +)?\)+', value)  # " == true )"
            if statement != None:
              statement = statement.group(0)

              # Check for brackets. Left bracket < Right bracket
              if statement.count('(') < statement.count(')'):

                delta = statement.count(')') - statement.count('(')
                fixed = value.replace(statement, ")"*delta)

                #print("Fixed to:", fixed)
                value = fixed
                metadata[0] = fixed
                code_lines[index] = fixed

                boolean_clean = False

            # Check inefficient bool check for the left side
            statement = re.search(r'\(+( +)?true( +)?(\)+)?( +)?==( +)?', value)  # "( true == "
            if statement != None:
              statement = statement.group(0)

              # Check for brackets. Left bracket > Right bracket
              if statement.count('(') > statement.count(')'):

                # Check how many brackets currently on the left
                delta = statement.count('(') - statement.count(')')
                fixed = value.replace(statement, "("*delta)

                #print("Fixed to:", fixed)
                value = fixed
                metadata[0] = fixed
                code_lines[index] = fixed

                boolean_clean = False

            # Check inefficient bool check for == true ==
            statement = re.search(r'(\(+)?( +)?==( +)?(\)+)?( +)?true( +)?(\(+)?( +)?==( +)?(\)+)?', value)  # " == true == "
            if statement != None:
              statement = statement.group(0)

              # Check for brackets. Left bracket == Right bracket
              if statement.count('(') == statement.count(')'):

                fixed = value.replace(statement, " == ")

                #print("Fixed to:", fixed)
                value = fixed
                metadata[0] = fixed
                code_lines[index] = fixed

                boolean_clean = False

            # Check inefficient bool check for " == false == " should not happen.

            # Check inefficient bool check for true == true  (out of research scope)
            statement = re.search(r'\(+( +)?true( +)?(\)+)?(( +)?==( +)?(\(+)?( +)?true( +)?\)+)+\)+', value)  # " ( true [== true ]) "
            if statement != None:
              statement = statement.group(0)

              # Check for brackets. Left bracket == Right bracket
              if statement.count('(') == statement.count(')'):

                fixed = value.replace(statement, "(true)")

                #print("Fixed to:", fixed)
                value = fixed
                metadata[0] = fixed
                code_lines[index] = fixed

                boolean_clean = False

            # Check inefficient bool check for false == false  (out of research scope)
            statement = re.search(r'\(+( +)?false( +)?(\)+)?(( +)?==( +)?(\(+)?( +)?false( +)?(\)+)?)+\)+', value)  # " ( false [== false ]) "
            if statement != None:
              statement = statement.group(0)

              # Check for brackets. Left bracket == Right bracket
              if statement.count('(') == statement.count(')'):

                fixed = value.replace(statement, "(false)")

                #print("Fixed to:", fixed)
                value = fixed
                metadata[0] = fixed
                code_lines[index] = fixed

                boolean_clean = False

      ## Index variables in event
      if INDEX_EVENT_VARIABLE == True:
        if token == 105:
          print(index, "index, event detected", value)

          # Check all variables in event - assume that the event is one line of code
          clean = re.sub("^(.*?)event(.*?)\(( +)?|( +)?\)( +)?;(.*$)?", "", value)  # Delete "...event x( " and " ) ;..."
          clean = re.split(", +?", clean)  # Split ",[ ]"
          indexed = sum("indexed" in s for s in clean)  # Count how many vars have been indexed

          # Add "indexed" to variables if not 3 variables are indexed and less than total variables. First come, first serve mechanic.
          if (indexed < 3) and (indexed < len(clean)):
            fixed = value
            for idx, var in enumerate(clean):
              if "indexed" not in var:
                # Check data type. Skip if string or bytes
                if INDEX_EVENT_STRING_BYTES == False:
                  var_type = var.split(" ", 1)
                  if "string" in var_type or "bytes" in var_type:
                    continue

                # Add "indexed" keyword
                split = var.rsplit(" ", 1)  # Split last occurance of whitespace only
                clean[idx] = split[0] + " indexed " + split[1]

                fixed = fixed.replace(var, split[0] + " indexed " + split[1])

                indexed += 1
                if indexed == 3:
                  break

            #print("fixed to:", fixed)
            value = fixed
            metadata[0] = fixed
            code_lines[index] = fixed

      ## Lazy precompute operation
      if PRECOMPUTE_VALUE == True:
        if token == 504:  # " = "
          print(index, "index, assign operator detected", value)

          if value.count("=") < 2:
            fixed = value

            # Clean the code line
            clean = re.sub("^(.*?)=( +)?|( +)?;(.*$)?", "", fixed)  # Delete "...= " and " ;..."

            # Evaluate per small splits (for cases of function arguments)
            splits = re.split('\(|\)|=|<|>|%|&|\||,', clean)
            splits = list(filter(None, splits))  # Delete empty elements

            for j in splits:
              try:
                evaluated = str(eval(j))
                fixed = fixed.replace(j, evaluated)
              except:
                #print("Fail 1")
                pass

            # Evaluate as a whole (for general case)
            try:
              fixed = str(eval(fixed))
            except:
              #print("Fail 2")
              pass

            #print("Result:", fixed)
            value = fixed
            metadata[0] = fixed
            code_lines[index] = fixed

      ## Scan variable islands and initial variable values
      if token == 301:  # bool
        print(index, "index, bool detected")

        # Get variable name and value
        _line_data["var_value"], _line_data["var_name"] = get_var_name_value(_line_data["value"], "false")

        # Check byte size
        size = re.search(r' bool(.*?) ', value).group(1)
        if size == "":
          byte_size = 1
        elif size.isdecimal() == False:
          byte_size = 33   # Variable size (array) is set to 33 by default because they can take up more than one page
        add_variable_island(byte_size, _line_data)  # Added the token variable to know what kind of var this is.

      elif token == 302:  # int
        print(index, "index, int detected")

        # Get variable name and value
        _line_data["var_value"], _line_data["var_name"] = get_var_name_value(_line_data["value"], "0")

        # Check byte size
        size = re.search(r' int(.*?) ', value).group(1)
        if size == "":
          size = 256
        elif size.isdecimal() == False:
          # Check int bit size
          _temp_split = size.split("[")
          if _temp_split[0].isdecimal() == False:
            size = 256
          else:
            size = 264
        byte_size = int(size)//8
        add_variable_island(byte_size, _line_data)

      elif token == 303:  # uint
        print(index, "index, uint detected")

        # Get variable name and value
        _line_data["var_value"], _line_data["var_name"] = get_var_name_value(_line_data["value"], "0")

        # Check byte size
        size = re.search(r' uint(.*?) ', value).group(1)
        if size == "":
          size = 256
        elif size.isdecimal() == False:
          # Check int bit size
          _temp_split = size.split("[")
          if _temp_split[0].isdecimal() == False:
            size = 256
          else:
            size = 264
        byte_size = int(size)//8
        add_variable_island(byte_size, _line_data)

      elif token == 304:  # bytes
        print(index, "index, bytes detected")

        # Get variable name and value
        _line_data["var_value"], _line_data["var_name"] = get_var_name_value(_line_data["value"], "0x0")

        # Check byte size
        byte_size = re.search(r' bytes(.*?) ', value).group(1)
        if byte_size.isdecimal() == False:
          _temp_split = byte_size.split("[")
          if _temp_split[0] == "32":
            byte_size = 32
          else:
            byte_size = 33
        add_variable_island(int(byte_size), _line_data)

      elif token == 305:  # address
        print(index, "index, address detected on index")

        # Get variable name and value
        _line_data["var_value"], _line_data["var_name"] = get_var_name_value(_line_data["value"], "0x0000000000000000000000000000000000000000")

        # Check byte size
        size = re.search(r' address(.*?) ', value).group(1)
        if size == "":
          byte_size = 20
        elif size.isdecimal() == False:
          byte_size = 33
        add_variable_island(byte_size, _line_data)

      elif token == 306:  # mapping
        print(index, "index, mapping detected")

        # Get variable name and value
        _line_data["var_value"], _line_data["var_name"] = None, None

        add_variable_island(33, _line_data)  # Note: This is very variable, so take this with heavy grain of salt on its efficiency

      elif token == 307:  # string
        print(index, "index, string detected")

        # Get variable name and value
        _line_data["var_value"], _line_data["var_name"] = get_var_name_value(_line_data["value"], "''")

        add_variable_island(32.5, _line_data)  # Observed that setting this to 32 bytes is more gas efficient than 33

  code_metadata[index] = metadata

### Optimize source code
print("")
# Variable optimization
print("Island count:", len(all_vars))
for index, value in enumerate(all_vars):
  # Precompute variables
  if PRECOMPUTE_VARIABLE_VALUE == True:
    print("Precompute vars!")

    # Fix all variable values first
    # 0. Delete variable from metadata variables if the value is used for increment / decrement
    print("\nStep 0...")
    del_variables = []
    for i in value["metadata"]["variables"]:
      var_value = value["metadata"]["variables"][i]

      # Eval value with numbers only
      splits = re.split('\(|\)| |/|\*|=|<|>|%|&|\|', var_value)
      splits = list(filter(None, splits))  # Delete empty elements

      for j in splits:
        if any(char in j for char in ("++", "--")):
          clean = re.sub("\+\+|--", "", j)
          del_variables.append(clean)

    for i in del_variables:
      del value["metadata"]["variables"][i]

    # 1. Substitute all possible variables
    # Don't touch value that has "++" or "--", should be deleted from the list if exists. (done while processing the var name or this loop)
    print("\nStep 1...")
    for i in value["metadata"]["variables"]:
      var_value = value["metadata"]["variables"][i]
      #print(i, "=", var_value)
      has_fix = True

      while has_fix == True:
        has_fix = False
        splits = re.split('\(|\)| |\+|-|/|\*|=|<|>|%|&|\||,', var_value)
        splits = list(filter(None, splits))  # Delete empty elements

        # Substitution
        for val in splits:
          if val in value["metadata"]["variables"]:
            var_value = var_value.replace(val, value["metadata"]["variables"][val])
            value["metadata"]["variables"][i] = value["metadata"]["variables"][i].replace(val, value["metadata"]["variables"][val])
            has_fix = True

      #print(i, "=", value["metadata"]["variables"][i])

    # 2. Evaluate all literals
    print("\nStep 2...")
    if PRECOMPUTE_VALUE == True:
      for i in value["metadata"]["variables"]:
        var_value = value["metadata"]["variables"][i]

        # Eval value with numbers only
        if all(char not in var_value for char in ("x", "'", '"', "true", "false")):
          # Evaluate per small splits (for cases of function arguments)
          splits = re.split('\(|\)|=|<|>|%|&|\||,', var_value)
          splits = list(filter(None, splits))  # Delete empty elements

          for j in splits:
            try:
              evaluated = str(eval(j))
              var_value = var_value.replace(j, evaluated)
            except:
              pass
          value["metadata"]["variables"][i] = var_value

          # Evaluate as a whole (for general case)
          try:
            value["metadata"]["variables"][i] = str(eval(var_value))
          except:
            pass

        #print(i, "=", value["metadata"]["variables"][i])
    else:
      print("Skipped.")

    # 3. Assign precomputed values into the variables
    print("\nStep 3...")
    vars = []
    for i in value["metadata"]["byte_sizes"]:  # Select all byte sizes
      for idx, val in enumerate(value[i]):  # Select all elements in byte size list
        # Replace if variable value is not None, exists in variables metadata, and equals to the real variable value
        if (val["var_value"] != None) and (val["var_name"] in value["metadata"]["variables"]) and (val["var_value"] in val["value"]):
          value[i][idx]["value"] = val["value"].replace(val["var_value"], value["metadata"]["variables"][val["var_name"]])

          # Put them in code_lines (this step is needed if variable packing is set to False)
          code_lines[value[i][idx]["index"]] = value[i][idx]["value"]
          #print(value[i][idx]["index"])

  # Variable packing mechanic
  if VARIABLE_PACKING == True:
    ## Variable Packing
    if VARIABLE_PACKING_BEST_FIT == True:
      # Sort the variable based on their byte size in reverse and split based on their byte size
      sorted_var_big = {}
      sorted_var_small = {}
      for i in sorted(value["metadata"]["byte_sizes"], reverse=True):
        if i <= 31:
          sorted_var_small[i] = value[i]
        else:
          sorted_var_big[i] = value[i]

      # Do best fit algorithm only for byte size <= 31
      optimized_var_small = bestFit(sorted_var_small)

      # Combine the big and optimized dictionaries into an array
      sorted_var_big = {**sorted_var_big, **optimized_var_small}
      sorted_var = []
      for i in sorted_var_big:
        sorted_var.extend(sorted_var_big[i])

      #print("sorted_var_big:", sorted_var_big)
      #print("sorted_var", sorted_var)

      # Organize source code to the optimized list
      for idx, val in enumerate(sorted_var):  # Paste the source code to the real list
        code_lines[value["metadata"]["start_index"]+idx] = val["value"]

    else:
      # Sort the variables based on their byte size in reverse
      # Reason to reverse order is to have larger byte size partitioned first, then the smaller byte size can fill in the remaining empty slots
      sorted_var = []
      for i in sorted(value["metadata"]["byte_sizes"], reverse=True):
        sorted_var.extend(value[i])

      #print("Sorted vars:", sorted_var)

      # Organize source code to be sorted
      #for idx, val in enumerate(sorted_var):  # Copy the source code in order
      #  sorted_var[idx] = code_lines[val]
      for idx, val in enumerate(sorted_var):  # Paste the source code to the real list
        code_lines[value["metadata"]["start_index"]+idx] = val["value"]

print("")

### Combine all source code lines back
code_optimized = '\n'.join(code_lines)

print("all_vars:", all_vars)
print("code_optimized:", code_optimized)
code_metadata

5 index, uint detected
6 index, address detected on index
7 index, address detected on index
8 index, mapping detected
9 index, default value int issue found!!   int public more = 0;
9 index, int detected
10 index, int detected
11 index, default value int issue found!!   int8 more2 =0;
11 index, int detected
12 index, uint detected
13 index, default value int issue found!!   uint8 more5=0;
13 index, uint detected
14 index, uint detected
15 index, int detected
16 index, int detected
17 index, address detected on index
18 index, string detected
19 index, default value bool issue found!!   bool bool1 = false;
19 index, bool detected
20 index, bool detected
21 index, default value string issue found!!   string str1 = "";
21 index, string detected
22 index, string detected
23 index, default value string issue found!!   string str3 = '';
23 index, string detected
24 index, default value address issue found!!   address address1 = 0x0000000000000000000000000000000000000000;
24 index, address d

{0: [''],
 1: ['// SPDX-License-Identifier: GPL-3.0'],
 2: ['pragma solidity ^0.4.24;'],
 3: [''],
 4: ['contract something {', 0, 100],
 5: ['  uint public coolade;', 303, 401],
 6: ['  address public checker;', 305, 401],
 7: ['  address public owner;', 305, 401],
 8: ['  mapping(address => bool) public whitelist;', 306, 401, 504],
 9: ['  int public more;', 202, 302, 401],
 10: ['  int256 public more3=23;', 302, 401, 504],
 11: ['  int8 more2;', 202, 302],
 12: ['  uint16 more4;', 303],
 13: ['  uint8 more5;', 202, 303],
 14: ['  uint[4] coolade2;', 303],
 15: ['  int[][2] coolade3;', 302],
 16: ['  int[2] coolade4;', 302],
 17: ['  address[] fishe;', 305],
 18: ['  string[] arr;', 307],
 19: ['  bool bool1;', 201, 301],
 20: ['  bool bool2 = true;', 301, 504],
 21: ['  string str1;', 203, 307],
 22: ['  string str2 = "aa";', 307, 504],
 23: ['  string str3;', 203, 307],
 24: ['  address address1;', 204, 305],
 25: ['  address address2 = 0x0000000000000000000000000000000000000123;',

In [10]:
# Output the optimized code to file
with open("output.sol", "w") as text_file:
    text_file.write(code_optimized)

## The Meat (https://regexr.com/)
### Regex
### https://github.com/andhikan207/SC_TestBench/blob/main/Benchmarking.ipynb
### https://pynative.com/python-regex-pattern-matching/
### https://www.regextutorial.org/regex-for-numbers-and-ranges.php
### Optimization Tips
### https://www.linkedin.com/pulse/solidity-gas-golfing-1-uint8-vs-uint256-sudeep-sagar/
### https://dev.to/javier123454321/solidity-gas-optimizations-pt-3-packing-structs-23f4

In [ ]:
from IPython.display import clear_output
import pandas as pd
import re

#File name
file_name = "dummyfile.csv"

In [ ]:
#Load smart contracts list.
df = pd.read_csv(file_name)

#Convert dataframe to list.
smartContracts = []
smartContracts2 = []
smartContracts = df['Source Code'].tolist()

#Process to add keywords to individual smart contracts.
unfinishedKW = "UNFINISHED_SC"
failedKW = "NOT_A_SC"

#Manually input keywords
numCycle1 = 1

for i in smartContracts:
  clear_output(wait = True)
  print(i + "\n")

  while True:
    print(f"{numCycle1}. Please check this smart contract. (1. Finished, 2. Unfinished, 3. Failed)")
    scCondition = input("Input a number: ")

    if scCondition == "1":
      smartContracts2.append(i)
      numCycle1 += 1
      break
    elif scCondition == "2":
      smartContracts2.append(unfinishedKW + " " + i)
      numCycle1 += 1
      break
    elif scCondition == "3":
      smartContracts2.append(failedKW + " " + i)
      numCycle1 += 1
      break
    else:
      print("\n> Wrong input. Try again!")

clear_output(wait = True)
print("Done, please proceed to the next step!")

Done, please proceed to the next step!


In [ ]:
#Choose ERC Criteria
clear_output(wait = True)
chooseERC = ""

while True:
  print("1. ERC-20")
  print("2. ERC-4626")
  print("3. ERC-3156")
  chooseERC = input("Which scenario would you choose?: ")

  if chooseERC == "1" or chooseERC == "2" or chooseERC == "3":
    break
  else:
    print("\n> Wrong input. Try again!")


#Set criteria weight
clear_output(wait = True)

cWeight1 = 0
cWeight2 = 0
cWeight3 = 0
cWeight4 = 0
cWeight5 = 0
cWeight6 = 0
cWeight7 = 0
cWeight8 = 0
cWeight9 = 0
cWeight10 = 0

criteriaDesc = ["Complete source code", "Packing variable", "Packing booleans", "Storage type", "Data type", "Variable size type", "Default value", "Functions", "Storage limiting", "Minimize on-chain data"]
cWeights = [cWeight1, cWeight2, cWeight3, cWeight4, cWeight5, cWeight6, cWeight7, cWeight8, cWeight9, cWeight10]

while True:
  totalPerc = 0
  numCycle2 = 1
  cArr = 0

  for i in criteriaDesc:
    while True:
      try:
        print(f"{numCycle2}. Please write score weight in PERCENTAGE for {i}. (Currently {totalPerc}%/100%)")
        setWeights = float(input("(Numbers only): "))

        cWeights[cArr] = 1 * (setWeights / 100)

        cArr += 1
        numCycle2 += 1
        totalPerc += setWeights
        clear_output(wait = True)
        break
      except ValueError:
        print("\n> That is not a number!")

  totalWeight = 0
  for i in cWeights:
    totalWeight += i

  if round(totalWeight) != 1.0:
    print("> Weight exceed 1.0 or below!\n")
    continue
  else:
    clear_output(wait = True)
    cArr = 0

    print("[Criteria Weight]\n")

    for i in criteriaDesc:
      print(f"{i}: {cWeights[cArr]}")
      cArr += 1

    print(f"\nTotal weight: {round(totalWeight)}\n")
    weightDone = input("Proceed? y/n: ")

    if weightDone == "n":
      clear_output(wait = True)
      continue
    else:
      break

clear_output(wait = True)
print("Done, please proceed to the next step!")

Done, please proceed to the next step!


In [ ]:
#Benchmarking
def benchmarkProgram():
  #REGEX PATTERNS
  criteria1_UFSC = unfinishedKW #Unfinished Source Code
  criteria2_ZONK = failedKW #Failed Source Code
  notPacked = r"(uint|uint(256))+ public [A-Za-z0-9]+ = [0-9]+;(\n)+\s+uint(8|16|32|128)" #Packing variable
  isBoolean = r"(uint|uint[0-9])+\([0-9]\)" #Packing boolean

  #Storage Type
  possibleC1_V1 = "uint"
  possibleC1_V2 = "uint256"
  pat7 = r"uint(?!(?:256))\d+ public" # uint*

  pat1 = r"mapping\([^)]*\) public [A-Za-z0-9]+;" #Mapping
  pat3 = r"[A-Za-z0-9]+\[\] public [A-Za-z0-9]+;" #Array

  fixA1 = r"(uint|string|int)\[[0-9]+\]" #Fixed
  pat6 = r"[A-Za-z0-9]+\[\]" #Dynamic array

  pat4 = r"public [A-Za-z0-9]+ = 0;" #Default value

  #Functions
  extfunc = r"function [A-Za-z0-9]+\([^)]*\) external" #External function
  pubfunc = r"function [A-Za-z0-9]+\([^)]*\) public" #Pub funct

  def findCriteria(w):
    return re.compile(r'\b({0})\b'.format(w), flags=re.IGNORECASE).search

  def checkCriteria(pattern):
    return re.compile(pattern, re.IGNORECASE).search


  #Init settings
  benchmark_score = 0.0

  indivExcessive = []
  indivScores = []

  perfect_score = 100.0
  partial_score = 50.0
  no_score = 0.0

  #Information Tracking
  completeSC1 = 0
  completeSC2 = 0
  completeSC3 = 0
  storageType1 = 0
  storageType2 = 0
  storageType3 = 0
  packingVar1 = 0
  packingVar2 = 0
  packingBool1 = 0
  packingBool2 = 0
  defaultVal1 = 0
  defaultVal2 = 0
  dataType1 = 0
  dataType2 = 0
  dataType3 = 0
  sizeType1 = 0
  sizeType2 = 0
  functionType1 = 0
  functionType2 = 0
  functionType3 = 0
  metCriteria = 0
  exceedData = 0

  numCycle5 = 0
  for sc in smartContracts2:
    excessiveData = 0
    temp_score = 0.0

    #Complete SC
    if findCriteria(criteria2_ZONK)(sc):
      completeSC1 += 1
      temp_score += no_score
    elif findCriteria(criteria1_UFSC)(sc):
      completeSC2 += 1
      temp_score += (partial_score * cWeights[0])
    else:
      completeSC3 += 1
      temp_score += (perfect_score * cWeights[0])

    #Packing Variables
    if checkCriteria(notPacked)(sc):
      packingVar1 += 1
      temp_score += no_score
    else:
      packingVar2 += 1
      temp_score += (perfect_score * cWeights[1])

    #Packing Boolean
    if checkCriteria(isBoolean)(sc):
      packingBool1 += 1
      temp_score += (perfect_score * cWeights[2])
    else:
      packingBool2 += 1
      temp_score += no_score

    #Storage type
    if checkCriteria(pat7)(sc) and findCriteria(possibleC1_V1)(sc) or findCriteria(possibleC1_V2)(sc): # Variety uint
      storageType1 += 1
      temp_score += (partial_score * cWeights[3])
    elif findCriteria(possibleC1_V1)(sc) or findCriteria(possibleC1_V2)(sc) and not checkCriteria(pat7)(sc): # uint256
      storageType2 += 1
      temp_score += no_score
    elif checkCriteria(pat7)(sc) and not findCriteria(possibleC1_V1)(sc) or findCriteria(possibleC1_V2)(sc): # uint*
      storageType3 += 1
      temp_score += (perfect_score * cWeights[3])

    #Mapping vs array
    if checkCriteria(pat1)(sc) and checkCriteria(pat3)(sc): # Mapping & Array
      dataType1 += 1
      temp_score += (partial_score * cWeights[4])
    elif checkCriteria(pat1)(sc) and not checkCriteria(pat3)(sc): # Mapping
      dataType2 += 1
      temp_score += (perfect_score * cWeights[4])
    elif checkCriteria(pat3)(sc) and not checkCriteria(pat1)(sc): # Array
      dataType3 += 1
      temp_score += no_score

    #Variable size type
    if checkCriteria(fixA1)(sc): # Fixed
      sizeType1 += 1
      temp_score += (perfect_score * cWeights[5])
    else: # Dynamic
      sizeType2 += 1
      temp_score += no_score

    #Default value
    if checkCriteria(pat4)(sc): # Initialized value
      defaultVal1 += 1
      temp_score += no_score
    else:
      defaultVal2 += 1
      temp_score += (perfect_score * cWeights[6]) # Default

    #Function
    if checkCriteria(extfunc)(sc) and checkCriteria(pubfunc)(sc): #mixed function
      functionType1 += 1
      temp_score += (partial_score * cWeights[7])
    elif checkCriteria(extfunc)(sc) and not checkCriteria(pubfunc)(sc): # only external
      functionType2 += 1
      temp_score += (perfect_score * cWeights[7])
    elif checkCriteria(pubfunc)(sc) and not checkCriteria(extfunc)(sc): # only public
      functionType3 += 1
      temp_score += no_score

    #Limit Storage & On-Chain data
    if chooseERC == "1":
      #ERC20 Criterias
      erc20_1 = "function name\([^)]*\)"
      erc20_2 = "function symbol\([^)]*\)"
      erc20_3 = "function decimal\([^)]*\)"
      erc20_4 = "function totalSupply\([^)]*\)"
      erc20_5 = "function balanceOf\([^)]*\)"
      erc20_6 = "function allowance\([^)]*\)"
      erc20_7 = "function transfer\([^)]*\)"
      erc20_8 = "function approve\([^)]*\)"
      erc20_9 = "function transferFrom\([^)]*\)"
      erc20_10 = "event Transfer\([^)]*\)"
      erc20_11 = "event Approval\([^)]*\)"

      offchains1 = "function (?!totalSupply|_totalSupply|balanceOf|_balanceOf|allowance|_allowance|transfer|_transfer|approve|_approve|transferFrom|_transferFrom|decimal|_decimal|symbol|_symbol|name|_name)\w+\([^)]*\)"
      offchains2 = "event (?!Approval|Transfer)\w+\([^)]*\)"

      tS = 0
      regexListSC1 = [erc20_1, erc20_2, erc20_3, erc20_4, erc20_5, erc20_6, erc20_7, erc20_8, erc20_9, erc20_10, erc20_11]

      erc20score = 100 / len(regexListSC1)
      ERC20criteriaMet = 0

      for testCase in regexListSC1:
        i = checkCriteria(testCase)(sc)

        if i:
          tS += erc20score
          ERC20criteriaMet += 1

      if ERC20criteriaMet == len(regexListSC1):
        metCriteria += 1

      #Check on Chains
      checkOffChain1 = len(re.findall(offchains1, sc, re.IGNORECASE))
      checkOffChain2 = len(re.findall(offchains2, sc, re.IGNORECASE))

      excessiveData = excessiveData + (checkOffChain1 + checkOffChain2)
      offChainScore = 100 - ((checkOffChain1 + checkOffChain2) * 10)

      if offChainScore < 0:
        offChainScore = 0
        exceedData += 1

      temp_score += (tS * cWeights[8])
      temp_score += (offChainScore * cWeights[9])

    elif chooseERC == "2":
      #ERC4626 Criterias
      erc4626_1 = "function asset\([^)]*\)"
      erc4626_2 = "function totalAssets\([^)]*\)"
      erc4626_3 = "function convertToShares\([^)]*\)"
      erc4626_4 = "function convertToAssets\([^)]*\)"
      erc4626_5 = "function maxDeposit\([^)]*\)"
      erc4626_6 = "function previewDeposit\([^)]*\)"
      erc4626_7 = "function deposit\([^)]*\)"
      erc4626_8 = "function maxMint\([^)]*\)"
      erc4626_9 = "function previewMint\([^)]*\)"
      erc4626_10 = "function mint\([^)]*\)"
      erc4626_11 = "function maxWithdraw\([^)]*\)"
      erc4626_12 = "function previewWithdraw\([^)]*\)"
      erc4626_13 = "function withdraw\([^)]*\)"
      erc4626_14 = "function maxRedeem\([^)]*\)"
      erc4626_15 = "function previewRedeem\([^)]*\)"
      erc4626_16 = "function redeem\([^)]*\)"
      erc4626_17 = "event Deposit\([^)]*\)"
      erc4626_18 = "event Withdraw\([^)]*\)"

      offchains1 = "function (?!asset|totalAssets|convertToShares|convertToAssets|maxDeposit|previewDeposit|deposit|maxMint|previewMint|mint|maxWithdraw|previewWithdraw|withdraw|maxRedeem|previewRedeem|redeem)\w+\([^)]*\)"
      offchains2 = "event (?!Deposit|Withdraw)\w+\([^)]*\)"

      tS = 0
      regexListSC2 = [erc4626_1, erc4626_2, erc4626_3, erc4626_4, erc4626_5, erc4626_6, erc4626_7, erc4626_8, erc4626_9, erc4626_10, erc4626_11, erc4626_12, erc4626_13, erc4626_14, erc4626_15, erc4626_16, erc4626_17, erc4626_18]

      erc4626score = 100 / len(regexListSC2)
      ERC4626criteriaMet = 0

      for testCase in regexListSC2:
        i = checkCriteria(testCase)(sc)

        if i:
          tS += erc4626score
          ERC4626criteriaMet += 1

      if ERC4626criteriaMet == len(regexListSC2):
        metCriteria += 1

      #Check on Chains
      checkOffChain1 = len(re.findall(offchains1, sc, re.IGNORECASE))
      checkOffChain2 = len(re.findall(offchains2, sc, re.IGNORECASE))

      excessiveData = excessiveData + (checkOffChain1 + checkOffChain2)
      offChainScore = 100 - ((checkOffChain1 + checkOffChain2) * 10)

      if offChainScore < 0:
        offChainScore = 0
        exceedData += 1

      temp_score += (tS * cWeights[8])
      temp_score += (offChainScore * cWeights[9])

    elif chooseERC == "3":
      erc3156_1 = "function maxFlashLoan\([^)]*\)"
      erc3156_2 = "function flashFee\([^)]*\)"
      erc3156_3 = "function flashLoan\([^)]*\)"
      erc3156_4 = "function onFlashLoan\([^)]*\)"

      offchains1 = "function (?!maxFlashLoan|flashFee|flashLoan|onFlashLoan)\w+\([^)]*\)"
      offchains2 = "event [A-Za-z0-9]+\([^)]*\)"

      tS = 0
      regexListSC3 = [erc3156_1, erc3156_2, erc3156_3, erc3156_4]

      erc3156score = 100 / len(regexListSC3)
      ERC3156criteriaMet = 0

      for testCase in regexListSC3:
        i = checkCriteria(testCase)(sc)

        if i:
          tS += erc3156score
          ERC3156criteriaMet += 1

      if ERC3156criteriaMet == len(regexListSC3):
        metCriteria += 1

      #Check on Chains
      checkOffChain1 = len(re.findall(offchains1, sc, re.IGNORECASE))
      checkOffChain2 = len(re.findall(offchains2, sc, re.IGNORECASE))

      excessiveData = excessiveData + (checkOffChain1 + checkOffChain2)
      offChainScore = 100 - ((checkOffChain1 + checkOffChain2) * 10)

      if offChainScore < 0:
        offChainScore = 0
        exceedData += 1

      temp_score += (tS * cWeights[8])
      temp_score += (offChainScore * cWeights[9])


    indivExcessive.append(excessiveData)
    indivScores.append(round(temp_score, 2))
    numCycle5 += 1

    #Set final score
    benchmark_score += temp_score

  #Information Output
  print(f"[BENCHMARKING {numCycle5} SMART CONTRACTS RESULT]\n")
  print("Individual excessive data count:")
  print(', '.join([str(x) for x in indivExcessive]))
  print("Individual score:")
  print(', '.join([str(x) for x in indivScores]))
  print("\n")

  print("Complete source code")
  print("==============================")
  print(f"Complete SC: {completeSC3}")
  print(f"Unfinished SC: {completeSC2}")
  print(f"Failed SC: {completeSC1}\n")
  print("Packing variable")
  print("==============================")
  print(f"Packed Variable: {packingVar2}")
  print(f"Unpacked Variable: {packingVar1}\n")
  print("Packing boolean")
  print("==============================")
  print(f"Packed Bool: {packingBool1}")
  print(f"Unpacked Bool: {packingBool2}\n")
  print("Storage type")
  print("==============================")
  print(f"uint*: {storageType3}")
  print(f"Mixed storage: {storageType1}")
  print(f"uint256: {storageType2}\n")
  print("Data type")
  print("==============================")
  print(f"Only mapping: {dataType2}")
  print(f"Mixed: {dataType1}")
  print(f"Only array: {dataType3}\n")
  print("Variable size type")
  print("==============================")
  print(f"Fixed: {sizeType1}")
  print(f"Dynamic: {sizeType2}\n")
  print("Default value")
  print("==============================")
  print(f"Default Value: {defaultVal2}")
  print(f"Initialized: {defaultVal1}\n")
  print("Functions")
  print("==============================")
  print(f"External: {functionType2}")
  print(f"Mixed: {functionType1}")
  print(f"Public: {functionType3}\n")
  print("Storage limiting")
  print("==============================")
  print(f"Met storage standard: {metCriteria}\n")
  excessiveData /= 20
  print("Minimize on-chain data")
  print("==============================")
  print(f"Number of SC that exceed maximum on-chain data: {exceedData}\n")

  #Calculate average score
  print("============================================================")
  print(f"Total score: {round(benchmark_score, 2)}")
  benchmark_score /= len(smartContracts)
  print(f"Average score: {round(benchmark_score, 2)}")

benchmarkProgram()

[BENCHMARKING 5 SMART CONTRACTS RESULT]

Individual excessive data count:
4, 2, 0, 0, 1
Individual score:
57.36, 48.0, 58.64, 49.55, 55.82


Complete source code
Complete SC: 5
Unfinished SC: 0
Failed SC: 0

Packing variable
Packed Variable: 5
Unpacked Variable: 0

Packing boolean
Packed Bool: 0
Unpacked Bool: 5

Storage type
uint*: 0
Mixed storage: 4
uint256: 1

Data type
Only mapping: 4
Mixed: 1
Only array: 0

Variable size type
Fixed: 0
Dynamic: 5

Default value
Default Value: 3
Initialized: 2

Functions
External: 1
Mixed: 1
Public: 3

Storage limiting
Met storage standard: 0

Minimize on-chain data
Number of SC that exceed maximum on-chain data: 0

Total score: 269.36
Average score: 53.87
